In [1]:
import pandas as pd
import csv
from loader import *
from pair_finder import *
import time
from random import seed
import keyboard
from random import seed
from random import randint

In [2]:
ratings_path_s = "DATA\\ratings_100users_shuffled.csv"
ratings_path = "DATA\\ratings_100users.csv"
movies_path =  "DATA\\movies.csv"

MinScore = 4
MaxCombo = 5
MinFrequency = 0.2
ratings_stream = createRatingsStream(ratings_path, MinScore)
userBaskets, movie_list = CreateMovieBaskets(ratings_path, MinScore, 100)
movies_df = ReadMovies(movies_path, movie_list)

#combos = myApriori(userBaskets, 0.3, 5)

In [3]:
mycombos = myApriori(userBaskets, MinFrequency, MaxCombo)

k: 3 User:  1 / 100
k: 3 User:  2 / 100
k: 3 User:  3 / 100
k: 3 User:  4 / 100
k: 3 User:  5 / 100
k: 3 User:  6 / 100
k: 3 User:  7 / 100
k: 3 User:  8 / 100
k: 3 User:  9 / 100
k: 3 User:  10 / 100
k: 3 User:  11 / 100
k: 3 User:  12 / 100
k: 3 User:  13 / 100
k: 3 User:  14 / 100
k: 3 User:  15 / 100
k: 3 User:  16 / 100
k: 3 User:  17 / 100
k: 3 User:  18 / 100
k: 3 User:  19 / 100
k: 3 User:  20 / 100
k: 3 User:  21 / 100
k: 3 User:  22 / 100
k: 3 User:  23 / 100
k: 3 User:  24 / 100
k: 3 User:  25 / 100
k: 3 User:  26 / 100
k: 3 User:  27 / 100
k: 3 User:  28 / 100
k: 3 User:  29 / 100
k: 3 User:  30 / 100
k: 3 User:  31 / 100
k: 3 User:  32 / 100
k: 3 User:  33 / 100
k: 3 User:  34 / 100
k: 3 User:  35 / 100
k: 3 User:  36 / 100
k: 3 User:  37 / 100
k: 3 User:  38 / 100
k: 3 User:  39 / 100
k: 3 User:  40 / 100
k: 3 User:  41 / 100
k: 3 User:  42 / 100
k: 3 User:  43 / 100
k: 3 User:  44 / 100
k: 3 User:  45 / 100
k: 3 User:  46 / 100
k: 3 User:  47 / 100
k: 3 User:  48 / 100
k

In [4]:
for i in range(1,8):
    print(len(mycombos[i]))

134
503
629
368
96
10
0


In [17]:
def sampledApriori(sample_size, ratings_stream, min_frequency, max_length):
    combos_1, key_stopped = run_apriori(sample_size, ratings_stream, min_frequency, max_length, key_enabled=True)
    
    if key_stopped:
        return combos_1
    
    combos_2, key_stopped = run_apriori(sample_size, ratings_stream, min_frequency, max_length)
    
    return [c1 for c1 in combos_1 if c1 in combos_2]


def run_apriori(sample_size, ratings_stream, min_frequency, max_length, key_enabled=False):
    SetOfUsers = [] # User so far
    sampleOfBaskets = {} # Key: userId, Value: movie_basket
    sample_map = [] # Mapping between sample index and userId

    key_pressed = False
    for i in range(len(ratings_stream)):

        if key_enabled and (keyboard.is_pressed('y') or keyboard.is_pressed('Y')) and i > len(ratings_stream):
            key_pressed = True
            break
        else:
            current_assessment = ratings_stream.iloc[i]
            current_user = int(current_assessment['userId'])
            current_movie = int(current_assessment['movieId'])

            if current_user not in SetOfUsers:
                SetOfUsers.append(current_user)
                sampleOfBaskets = reservoir_sampling(len(SetOfUsers), sample_size, sampleOfBaskets, sample_map, current_user)

            if current_user in sampleOfBaskets.keys():
                sampleOfBaskets[current_user].append(current_movie)
    
    combos_dict = myApriori(list(sampleOfBaskets.values()), min_frequency, max_length)
    combos = export_combos(combos_dict)


    return combos, key_pressed


def reservoir_sampling(n_distinct_users, sample_size, sample_of_baskets, sample_map, current_user):
    seed()
    if n_distinct_users <= sample_size:
        sample_of_baskets[current_user] = []
        sample_map.append(current_user)
    else:
        tmp = randint(0, n_distinct_users)

        if tmp < sample_size:
            old_user = sample_map[tmp]
            sample_map[tmp] = current_user
            del sample_of_baskets[old_user]
            sample_of_baskets[current_user] = []
            
    return sample_of_baskets


def export_combos(combos_dict):
    combos = []
    for i in combos_dict.keys():
        for j in combos_dict[i].keys():
            combos.append(j)
    return combos

In [18]:
combos= sampledApriori(50, ratings_stream, MinFrequency, MaxCombo)

k: 3 User:  1 / 50
k: 3 User:  2 / 50
k: 3 User:  3 / 50
k: 3 User:  4 / 50
k: 3 User:  5 / 50
k: 3 User:  6 / 50
k: 3 User:  7 / 50
k: 3 User:  8 / 50
k: 3 User:  9 / 50
k: 3 User:  10 / 50
k: 3 User:  11 / 50
k: 3 User: 12 / 50
k: 3 User:  13 / 50
k: 3 User:  14 / 50
k: 3 User:  15 / 50
k: 3 User:  16 / 50
k: 3 User:  17 / 50
k: 3 User:  18 / 50
k: 3 User:  19 / 50
k: 3 User:  20 / 50
k: 3 User:  21 / 50
k: 3 User:  22 / 50
k: 3 User:  23 / 50
k: 3 User:  24 / 50
k: 3 User:  25 / 50
k: 3 User:  26 / 50
k: 3 User:  27 / 50
k: 3 User:  28 / 50
k: 3 User:  29 / 50
k: 3 User:  30 / 50
k: 3 User:  31 / 50
k: 3 User:  32 / 50
k: 3 User:  33 / 50
k: 3 User:  34 / 50
k: 3 User:  35 / 50
k: 3 User:  36 / 50
k: 3 User:  37 / 50
k: 3 User:  38 / 50
k: 3 User:  39 / 50
k: 3 User:  40 / 50
k: 3 User:  41 / 50
k: 3 User:  42 / 50
k: 3 User:  43 / 50
k: 3 User:  44 / 50
k: 3 User:  45 / 50
k: 3 User:  46 / 50
k: 3 User:  47 / 50
k: 3 User:  48 / 50
k: 3 User: 49 / 50
k: 3 User:  50 / 50
k: 4 User: 

In [19]:
len(combos)

6280

In [1]:
mycombos_f = []
for i in mycombos.keys():
    for j in mycombos[i].keys():
        mycombos_f.append(j)

NameError: name 'mycombos' is not defined

In [2]:
tp = []
fp = []
fn = []
for i in mycombos_f:
    if i in combos:
        tp.append(i)
    else:
        fn.append(i)
for i in combos:
    if i not in mycombos_f:
        fp.append(i)
tp = len(tp)
fp = len(fp)
fn = len(fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * (precision * recall) / (precision + recall)

NameError: name 'combos' is not defined

In [ ]:
print(precision, recall, f1)

In [ ]:
x = {}
k = (1,2,3)
x[k] = 4

In [ ]:
1 in (1,2,3)

In [ ]:
p = [(1,2,3), (1,2,3), (2,3,4)]
set(p)